# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for E. Coli

# Imports

In [14]:
# from sklearnex import patch_sklearn
# patch_sklearn()

import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone

import matplotlib.pyplot as plt

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [15]:
ORGANISM = "ecoli"
TAX_ID = 83333
LOG_FILE = f"../logs/{ORGANISM}_amino_sugar.log"
N_THREADS = 16
IDENTITY_THRESHOLD=70
OUTLIERS = [
    "P0AAG8",
    "P04983",
    "P10346",
    "P28635",
    "P14175",
    "P69797",
    "P76773",
    "P37388",
    "P30750",
    "Q47706",
    "P10907",
    "P37774",
    "P0AAF3",
    "P64550",
    "P02943",
    "P75733",
    "P0AAF6",
    "P68187",
    "P69856",
    "P07109",
    "Q6BEX0",
]



# Dataset

In [16]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    outliers=OUTLIERS,
    verbose=True,
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P69801,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,manY pel ptsP b1818 JW1807,PTS system mannose-specific EIIC component (EI...,4.A.6.1.1,83333,MEITTLQIVLVFIVACIAGMGSILDEFQFHRPLIACTLVGIVLGDM...
P36672,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,treB b4240 JW4199,PTS system trehalose-specific EIIBC component ...,4.A.1.2.4,83333,MMSKINQTDIDRLIELVGGRGNIATVSHCITRLRFVLNQPANARPK...
P56580,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,srlE gutA gutE b2703 JW5430,PTS system glucitol/sorbitol-specific EIIB com...,4.A.4.1.1,83333,MTHIRIEKGTGGWGGPLELKATPGKKIVYITAGTRPAIVDKLAQLT...
P0AA47,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,plaP yeeF b2014 JW5330,Low-affinity putrescine importer PlaP,2.A.3.1.14,83333,MSHNVTPNTSRVELRKTLTLVPVVMMGLAYMQPMTLFDTFGIVSGL...
P08722,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,bglF bglC bglS b3722 JW3700,PTS system beta-glucoside-specific EIIBCA comp...,4.A.1.2.2,83333,MTELARKIVAGVGGADNIVSLMHCATRLRFKLKDESKAQAEVLKKT...
...,...,...,...,...,...,...,...,...
P19642,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,malX b1621 JW1613,PTS system maltose-specific EIICB component [I...,4.A.1.1.3,83333,MTAKTAPKVTLWEFFQQLGKTFMLPVALLSFCGIMLGIGSSLSSHD...
P0AAD4,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,tyrP b1907 JW1895,Tyrosine-specific transport protein (Tyrosine ...,2.A.42.1.1,83333,MKNRTLGSVFIVAGTTIGAGMLAMPLAAAGVGFSVTLILLIGLWAL...
P23173,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,tnaB trpP b3709 JW5619/JW5622,Low affinity tryptophan permease,2.A.42.1.3,83333,MTDQAEKKHSAFWGVMVIAGTVIGGGMFALPVDLAGAWFFWGAFIL...


## Clustering

In [17]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [18]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P69801,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,manY pel ptsP b1818 JW1807,PTS system mannose-specific EIIC component (EI...,4.A.6.1.1,83333,MEITTLQIVLVFIVACIAGMGSILDEFQFHRPLIACTLVGIVLGDM...
P36672,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,treB b4240 JW4199,PTS system trehalose-specific EIIBC component ...,4.A.1.2.4,83333,MMSKINQTDIDRLIELVGGRGNIATVSHCITRLRFVLNQPANARPK...
P56580,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,srlE gutA gutE b2703 JW5430,PTS system glucitol/sorbitol-specific EIIB com...,4.A.4.1.1,83333,MTHIRIEKGTGGWGGPLELKATPGKKIVYITAGTRPAIVDKLAQLT...
P0AA47,Amino-acid transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,plaP yeeF b2014 JW5330,Low-affinity putrescine importer PlaP,2.A.3.1.14,83333,MSHNVTPNTSRVELRKTLTLVPVVMMGLAYMQPMTLFDTFGIVSGL...
P08722,Sugar transport,Cell inner membrane;Cell membrane;Membrane;Tra...,Transport,bglF bglC bglS b3722 JW3700,PTS system beta-glucoside-specific EIIBCA comp...,4.A.1.2.2,83333,MTELARKIVAGVGGADNIVSLMHCATRLRFKLKDESKAQAEVLKKT...


# Feature generation

## Labels

In [19]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_labels.labels.value_counts()

Amino-acid transport    51
Sugar transport         47
Name: labels, dtype: int64

## PAAC

In [20]:
calculate_composition_feature(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    feature_type="paac",
)

df_paac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    index_col=0,
)

# Functions

In [21]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [22]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy()
    sample_names = df_features.index.to_numpy()
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [23]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Individual Features

## PAAC

### Stats

In [24]:
df_stats = get_feature_stats(df_paac, df_labels).head(10)
# 10 best features
display(df_stats.head(10))

display(df_stats.sort_values("corr", ascending=False))

,corr,mean,std,corr_abs,mean0,mean1,median0,median1,mediandiff
LV,-0.289005,0.012882,0.006820,0.289005,0.014765,0.010840,0.014286,0.009174,0.005111
LA,-0.249625,0.015502,0.006920,0.249625,0.017151,0.013711,0.016878,0.012931,0.003947
GI,0.265128,0.009684,0.005651,0.265128,0.008253,0.011236,0.006849,0.010654,0.003805
AG,0.339882,0.011289,0.005819,0.339882,0.009400,0.013339,0.008772,0.012121,0.003349
GA,0.134654,0.009811,0.005319,0.134654,0.009127,0.010553,0.007500,0.010823,0.003323
AS,-0.263732,0.006539,0.003698,0.263732,0.007471,0.005529,0.007890,0.004831,0.003059
LD,0.126261,0.002526,0.002869,0.126261,0.002180,0.002901,0.000000,0.002941,0.002941
GG,0.281708,0.007726,0.004958,0.281708,0.006392,0.009173,0.006494,0.009434,0.002940
AL,-0.244240,0.016697,0.008420,0.244240,0.018661,0.014565,0.017316,0.014423,0.002893
VV,-0.107337,0.007658,0.004869,0.107337,0.008157,0.007116,0.009050,0.006289,0.002760


,corr,mean,std,corr_abs,mean0,mean1,median0,median1,mediandiff
AG,0.339882,0.011289,0.005819,0.339882,0.009400,0.013339,0.008772,0.012121,0.003349
GG,0.281708,0.007726,0.004958,0.281708,0.006392,0.009173,0.006494,0.009434,0.002940
GI,0.265128,0.009684,0.005651,0.265128,0.008253,0.011236,0.006849,0.010654,0.003805
GA,0.134654,0.009811,0.005319,0.134654,0.009127,0.010553,0.007500,0.010823,0.003323
LD,0.126261,0.002526,0.002869,0.126261,0.002180,0.002901,0.000000,0.002941,0.002941
VV,-0.107337,0.007658,0.004869,0.107337,0.008157,0.007116,0.009050,0.006289,0.002760
AL,-0.244240,0.016697,0.008420,0.244240,0.018661,0.014565,0.017316,0.014423,0.002893
LA,-0.249625,0.015502,0.006920,0.249625,0.017151,0.013711,0.016878,0.012931,0.003947
AS,-0.263732,0.006539,0.003698,0.263732,0.007471,0.005529,0.007890,0.004831,0.003059
LV,-0.289005,0.012882,0.006820,0.289005,0.014765,0.010840,0.014286,0.009174,0.005111


### Independent test set

In [25]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_paac, df_labels)

### Model selection

In [26]:
for estimator in [
    LinearSVC(random_state=0),
    LinearSVC(class_weight="balanced", random_state=0),
    SVC(),
    SVC(class_weight="balanced"),
    GaussianNB(),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=0),
    SGDClassifier(random_state=0),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(estimator, X_train, y_train, scoring="f1_macro")
    print("### ", type(estimator).__name__)
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC
CV folds: [0.36  0.333 0.333 0.348 0.348]
Mean: 0.344
Std: 0.01
###  LinearSVC
CV folds: [0.806 0.654 0.806 0.641 0.598]
Mean: 0.701
Std: 0.088
###  SVC
CV folds: [0.867 0.806 0.746 0.55  0.661]
Mean: 0.726
Std: 0.111
###  SVC
CV folds: [0.746 0.654 0.806 0.525 0.667]
Mean: 0.679
Std: 0.095
###  GaussianNB
CV folds: [0.36  0.746 0.619 0.498 0.641]
Mean: 0.573
Std: 0.132
###  KNeighborsClassifier
CV folds: [0.619 0.875 0.873 0.533 0.667]
Mean: 0.713
Std: 0.138
###  RandomForestClassifier
CV folds: [0.619 0.873 0.806 0.661 0.583]
Mean: 0.708
Std: 0.112
###  SGDClassifier
CV folds: [0.304 0.333 0.333 0.348 0.348]
Mean: 0.333
Std: 0.016


### Hyperparameters

In [27]:
# params = {
#     "kneighborsclassifier__n_neighbors": list(range(3,10)),
#     "kneighborsclassifier__weights": ["uniform", "distance"],
#     "kneighborsclassifier__metric": ["minkowski", "euclidean", "manhattan"],
# }
params = {
    "svc__C": [0.1, 1, 10,],
    "svc__gamma": ["scale", "auto", 1e-0, 1e-1, 1e-2],
    "svc__class_weight": ["balanced", None],
}
gsearch = GridSearchCV(
    make_pipeline(StandardScaler(), SVC()),
    param_grid=params,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale'}
0.7046732026143792


### Dimensionality reduction

In [28]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 95% variance:", np.argmax(csum >= 0.95) + 1)

gsearch = GridSearchCV(
    estimator=Pipeline(
        [
            ("scale", StandardScaler()),
            ("pca", PCA()),
            ("scale2", StandardScaler()),
            ("svc", SVC()),
        ]
    ),
    param_grid={
        "svc__C": [0.1, 1, 10],
        "svc__gamma": [1e-0, 1e-1, 1e-2, 1e-3,"scale"],
        "svc__class_weight": ["balanced", None],
        "pca__n_components": np.linspace(0.8, 0.99, 18),
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-2,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_.round(3))
best_estimator_svc_pca = gsearch.best_estimator_


Number of components to explain 95% variance: 62
{'pca__n_components': 0.833529411764706, 'svc__C': 1, 'svc__class_weight': None, 'svc__gamma': 0.01}
0.79


### Validation

In [29]:
best_estimator = best_estimator_svc_pca

In [30]:
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.79+-0.081


,Amino,Sugar,Macro,Weighted
precision,0.818,0.889,0.854,0.854
recall,0.900,0.800,0.850,0.850
f1-score,0.857,0.842,0.850,0.850
support,10.000,10.000,20.000,20.000


,Amino,Sugar
Amino,9,1
Sugar,2,8
